In [1]:
# Import library-library
import os
import glob
from tqdm import tqdm
import numpy as np

# Data Preparation and Preprocessing
import pandas as pd
import re
from string import digits

# Word Embedding
import joblib
from keybert import KeyBERT
kw_extractor = KeyBERT('distilbert-base-nli-mean-tokens')
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('xlm-r-distilroberta-base-paraphrase-v1')

# Input and Expansion Query
import nltk
nltk.download('punkt')
import math
from textblob import TextBlob
from yake import KeywordExtractor
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk import tokenize
from operator import itemgetter
from nltk.tokenize import word_tokenize
from textblob import TextBlob
NLTK_StopWords = stopwords.words('indonesian')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def preprocessing(berita):
    s = berita.lower()
    s = s.replace('\n', ' ')
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    tokens = [token for token in s.split(" ") if token != ""]
    T = [t for t in tokens if t not in NLTK_StopWords]
    return T

In [3]:
df_test = pd.read_csv('df_test.csv')
df_test = df_test[pd.notnull(df_test['description'])]
print(df_test.info())
print ('-'*90)
document_text_test= joblib.load('document_text_test.pkl')
print(len(document_text_test))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159 entries, 0 to 158
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        159 non-null    object
 1   date         159 non-null    object
 2   description  159 non-null    object
 3   source       159 non-null    object
dtypes: object(4)
memory usage: 6.2+ KB
None
------------------------------------------------------------------------------------------
159


In [4]:
df_train = pd.read_csv('df_train.csv')
df_train = df_train[pd.notnull(df_train['description'])]
print(df_train.info())
print ('-'*90)
document_text_train= joblib.load('document_text_train.pkl')
print(len(document_text_train))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1428 entries, 0 to 1428
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        1428 non-null   object
 1   date         1428 non-null   object
 2   description  1428 non-null   object
 3   source       1428 non-null   object
dtypes: object(4)
memory usage: 55.8+ KB
None
------------------------------------------------------------------------------------------
1429


In [5]:
bow_kecelakaan_read = pd.read_csv('bow_kecelakaan.csv')
bow_kecelakaan_read.head()

,tingkat setelah parent,parent,similarity
0,1,kecelakaan,"[('musibah', 0.7218754887580872), ('insiden', ..."
1,2,musibah,"[('kecelakaan', 0.7218754291534424), ('insiden..."
2,2,insiden,"[('kecelakaan', 0.7198260426521301), ('peristi..."
3,2,kecelakan,"[('kecelakaan', 0.6681101322174072), ('dekompr..."
4,2,tabrakan,"[('ledakan', 0.6787223815917969), ('benturan',..."


In [6]:
bow_kecelakaan_text = []

for i in range(0, bow_kecelakaan_read.shape[0]):
    bow_kecelakaan_text.append(bow_kecelakaan_read.iloc[i,1])

In [7]:
bow_where_read = pd.read_csv('bow_where.csv')
bow_where_text = []

for i in range(0, bow_where_read.shape[0]):
    bow_where_text.append(bow_where_read.iloc[i,1])

In [8]:
bow_when_read = pd.read_csv('bow_when.csv')
bow_when_text = []

for i in range(0, bow_when_read.shape[0]):
    bow_when_text.append(bow_when_read.iloc[i,1])

In [9]:
bow_who_read = pd.read_csv('bow_where.csv')
bow_who_text = []

for i in range(0, bow_who_read.shape[0]):
    bow_who_text.append(bow_who_read.iloc[i,1])

In [10]:
def cari_dokpertama(kueriAsli):
    kueriPre=preprocessing(kueriAsli)
    kueriPre= " ".join (kueriPre)
    hasilSearch=[]
    tfidf_train = joblib.load('tfidf_train.pkl')
    tfidf_vectorizer = joblib.load('vectorizer.pkl')
    query_vec= tfidf_vectorizer.transform([kueriPre])
    # print('queryvec')
    print(query_vec)
    results=cosine_similarity(tfidf_train, query_vec).reshape((-1))
    # print(results)
    for i in results.argsort()[-5:][::-1]:
        hasilSearch.append(df_train.iloc[i,-2])
    hasilSearch=". ".join(hasilSearch)
    return hasilSearch



In [11]:
##Keywords Extraction with YAKE
def keyword_yake(hasilSearch):
    keywordYake=[]

    k_extractor = KeywordExtractor(lan="id", n=1, top=10)
    k_extractor2 = KeywordExtractor(lan="id", n=2, top=10)
    keywords = k_extractor.extract_keywords(text=hasilSearch)
    keywords = k_extractor2.extract_keywords(text=hasilSearch)
    keywordYake = [x for x, y in keywords]
    #keywordYake.append(keywords)
    #print (keywordYake)
    return keywordYake
#print("Keywords of article\n", keywords)

In [12]:
#Keywords Extraction with TFIDF
def keyword_tfidf(hasilSearch):

    keywordtfidf=[]
    keywordtfidf2=[]

    #doc = 'بَاب فرض الْوضُوء وسننه وهيآته وَفرض الْوضُوء سِتّ خِصَال النِّيَّة عمند غسل الْوَجْه وَغسل الْوَجْه وَغسل الذراعين مَعَ الْمرْفقين وَمسح مَا قل من الرَّأْس وَغسل الرجلَيْن مَعَ الْكَعْبَيْنِ وَالتَّرْتِيب وعَلى قَول الْوَلَاء وسننه عشر خِصَال خمس مِنْهَا قبل غسل الْوَجْه وَهِي التَّسْمِيَة وَغسل الْكَفَّيْنِ والمضمضة وَالِاسْتِنْشَاق وَالْمُبَالغَة فيههما إِلَّا للصَّائِم وَخمْس بعد غسل الْوَجْه وَهِي تَقْدِيم الْيُمْنَى على ليسرى وَمسح جَمِيع الرَّأْس وَمسح الْأُذُنَيْنِ ظاهرهما وباطنهما وَإِدْخَال الأصبعين فيهمَا وتخليل أَصَابِع الرجلَيْن . وَغسل دَاخل الْكَعْبَيْنِ وَلَيْسَ مسح لعنق من سنَنه وفضيلته تكراره ثَلَاثًا وزالواجب فِيهِ مرّة والمرتان أفضل وَالثَّلَاث أكمل وهيآته أَن يبْدَأ فِي تَطْهِير الْأَعْضَاء بمواضع الِابْتِدَاء . فَإِن اقْتصر على فروضه استة أَجزَأَهُ وَإِن ضيع حَظّ نَفسه فِيمَا ترك'
    total_words = re.sub(r'[^\w]', ' ', hasilSearch)
    total_words = total_words.lower().split()
    #print (total_words)
    total_word_length = len(total_words)
    total_sentences = tokenize.sent_tokenize(hasilSearch)
    total_sent_len = len(total_sentences)

    tf_score = {}
    for each_word in total_words:
        #print (each_word)
        each_word = each_word.replace('.','')
        if each_word not in NLTK_StopWords:
            if each_word in tf_score:
                tf_score[each_word] += 1
            else:
                tf_score[each_word] = 1

    # Dividing by total_word_length for each dictionary element
    tf_score.update((x, y/int(total_word_length)) for x, y in tf_score.items())
    #print(tf_score)
    def check_sent(word, sentences): 
        final = [all([w in x for w in word]) for x in sentences] 
        sent_len = [sentences[i] for i in range(0, len(final)) if final[i]]
        return int(len(sent_len))

    idf_score = {}
    for each_word in total_words:
        #print (each_word)
        each_word = each_word.replace('.','')
        if each_word not in NLTK_StopWords:
            if each_word in idf_score:
                idf_score[each_word] = check_sent(each_word, total_sentences)
            else:
                idf_score[each_word] = 1

    # Performing a log and divide
    idf_score.update((x, math.log(int(total_sent_len)/y)) for x, y in idf_score.items())

    #print(idf_score)
    tf_idf_score = {key: tf_score[key] * idf_score.get(key, 0) for key in tf_score.keys()}
    #print(tf_idf_score)
    def get_top_n(dict_elem, n):
        result = dict(sorted(dict_elem.items(), key = itemgetter(1), reverse = True)[:n]) 
        hasil =list(result.keys())
        #print(list(result.keys()))        
        return hasil
    #print(get_top_n(tf_idf_score, 25))
    #print(len(get_top_n(tf_idf_score, 1)))
    keywordtfidf.append(get_top_n(tf_idf_score, 25))
    for i in range(len(keywordtfidf)):
        #print (i)
        totalKw=0
        totalKw=len(keywordtfidf[i])
        for j in range(totalKw):
            #print (j)
            keywordtfidf2.append(keywordtfidf[i][j])
    #print (keywordtfidf2)
    return keywordtfidf2

In [13]:
#Keywords Extraction with BERT
def keyword_bert(hasilSearch):

    keywordbert=[]

    #for j in range(len(array_text)):
    keyword1 = kw_extractor.extract_keywords(hasilSearch, top_n=10, keyphrase_ngram_range=(1, 1))
    keyword2 = kw_extractor.extract_keywords(hasilSearch, top_n=10, keyphrase_ngram_range=(1, 2))

    #print("Keywords of article\n", keywords)
    for i in range (0,len (keyword1)):
        keywordbert.append(keyword1[i][0])
        keywordbert.append(keyword2[i][0])
    #print (keywordbert)
    return keywordbert

In [14]:
def rangking (keywordGabung):
    keywordTemp=[]
    keywordFinal=[]

    def borda_sort(lists):
        scores = {}
        for l in lists:
            for idx, elem in enumerate(reversed(l)):
                if not elem in scores:
                    scores[elem] = 0
                scores[elem] += idx
        return sorted(scores.keys(), key=lambda elem: scores[elem], reverse=True)

    keywordTemp.append(borda_sort(keywordGabung))
    print ('kandidat temp',keywordTemp)
    print ('Total Kandidat temp: ',len(keywordTemp[0]))

    if len(keywordTemp[0])>50:
        print ('kurang dari 50')
        for i in range (0,50):
            keywordFinal.append(keywordTemp[0][i])
    elif len(keywordTemp[0])<50:
        print ('lebih dari 50')
        for i in range (0,len(keywordTemp)):
            for j in range (0,len(keywordTemp[0])):
                keywordFinal.append(keywordTemp[0][j])
    print ('Total Kandidat final: ',len(keywordFinal))
    print ('Kandidat final: ',keywordFinal)
    return keywordFinal


In [15]:
def keyword_BOW(keywordBOW, kueriAsli):
    cekDuplicate = []
    kandidatFix = []

    for i in keywordBOW:
        if(i not in cekDuplicate and i!=0):
            cekDuplicate.append(i)

    queries=[kueriAsli]
    query_embeddings = embedder.encode(queries)
    corpus_embeddings4 = embedder.encode(cekDuplicate)
    
    # Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
    closest_n = 10
    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings4, 'cosine')[0]
        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])
        for idx, distance in results[0:closest_n]:
            kandidatFix.append(cekDuplicate[idx])
    return kandidatFix

In [16]:
#kueri what
kueriAsliWhat='apa sebenarnya kejadian yang terjadi diberita tersebut'
kueriWhat=preprocessing(kueriAsliWhat)
kueriWhat= [" ".join (kueriWhat)]
print (kueriWhat)
hasilkandidatWhat=[]
keywordGabungWhat=[]
kandidatFixWhat=[]
kueriFixWhat=[]
hasilDokumenWhat=[]


hasilSearchWhat=cari_dokpertama(kueriAsliWhat)
keywordYakeWhat=keyword_yake(hasilSearchWhat)
keywordtfidf2What=keyword_tfidf(hasilSearchWhat)
keywordbertWhat=keyword_bert (hasilSearchWhat)
keywordBOW_What=keyword_BOW(bow_kecelakaan_text, kueriAsliWhat)

keywordGabungWhat.append(keywordYakeWhat)
keywordGabungWhat.append(keywordtfidf2What)
keywordGabungWhat.append(keywordbertWhat)
keywordGabungWhat.append(keywordBOW_What)
hasilrankWhat=rangking(keywordGabungWhat)

for i in hasilrankWhat:
    kueriFixWhat.append(i)
for j in kueriFixWhat:
    hasilkandidatWhat.append(j)
kueriFixWhat=[preprocessing(i) for i in kueriFixWhat]
for i in kueriFixWhat:
    for j in i:
        kandidatFixWhat.append(j)
kandidatFixWhat= [" ".join (kandidatFixWhat)]
print ('*'*120)


['kejadian diberita']
  (0, 6763)	1.0
kandidat temp [['mengungkap', '1102', '15', 'terluka', 'kebayoran', 'armada', 'kabupaten', 'melaju', 'kabupaten majalengka', 'kencang', 'mengedepankan', 'menaikkan', 'masyarakat mengedepankan', 'jiwa', 'kecamatan', 'istri', 'kertajati kabupaten', 'irjen', 'mengakibatkan', 'boy', 'kecelakaan mengakibatkan', 'rafli', 'bantuan', 'amar', 'transjakarta menyebut', 'jasa raharja', 'bernopol', 'tmcpoldametro', 'terjadinya', 'korban', 'khusus', 'menjadwalkan pemeriksaan', 'dialaminya', 'penumpang mobil', 'akibat', 'tergabung', 'kejadiannya', 'luka luka', 'resmi', 'langsung dievakuasi', 'alhasil', 'kecelakaan', 'tmc', 'kisahnya', 'pajero terluka', 'polda', 'dugaan pemukulan', 'menimpanya', 'pajero istri', 'jaya', 'majalengka', 'berlakunya', 'penyebab kecelakaan', 'senin', 'menyebut armada', 'penyebabnya', 'mobil pajero', '7', 'keluarga', 'ditimpakan', 'kejadian', '2', 'transjakarta mengungkap', 'ceritanya']]
Total Kandidat temp:  64
kurang dari 50
Total Kand

In [17]:
#kueri where
kueriAsliWhere='di daerah mana kejadian itu terjadi'
kueriWhere=preprocessing(kueriAsliWhere)
kueriWhere= [" ".join (kueriWhere)]
print (kueriWhere)
hasilkandidatWhere=[]
keywordGabungWhere=[]
kandidatFixWhere=[]
kueriFixWhere=[]
hasilDokumenWhere=[]


hasilSearchWhere=cari_dokpertama(kueriAsliWhere)
keywordYakeWhere=keyword_yake(hasilSearchWhere)
keywordtfidf2Where=keyword_tfidf(hasilSearchWhere)
keywordbertWhere=keyword_bert (hasilSearchWhere)
keywordBOW_Where=keyword_BOW(bow_where_text, kueriAsliWhere)

keywordGabungWhere.append(keywordYakeWhere)
keywordGabungWhere.append(keywordtfidf2Where)
keywordGabungWhere.append(keywordbertWhere)
keywordGabungWhere.append(keywordBOW_Where)
hasilrankWhere=rangking(keywordGabungWhere)

for i in hasilrankWhere:
    kueriFixWhere.append(i)
for j in kueriFixWhere:
    hasilkandidatWhere.append(j)
kueriFixWhere=[preprocessing(i) for i in kueriFixWhere]
for i in kueriFixWhere:
    for j in i:
        kandidatFixWhere.append(j)
kandidatFixWhere= [" ".join (kandidatFixWhere)]
print ('*'*120)


['daerah kejadian']
  (0, 6763)	0.4340738310483123
  (0, 3216)	0.9008772997467753
kandidat temp [['iqbal', '18', 'kepergian', 'lumintang', 'anak', 'namanya', 'kabupaten', 'alika', 'bleberan kabupaten', '6', 'kekuatan', 'tk', 'kecamatan baleendah', 'abizar', 'keluarga', 'suami', 'kronologi kecelakaan', 'beji', 'mengakibatkan', 'depok', 'baleendah kecamatan', 'barat', 'keberadaan', '26', 'kecelakaan mengakibatkan', 'berangkat kerja', '2020', 'kecamatan', 'temanggung', 'pasar minggu', 'pamit', 'kabupaten cilacap', 'kuin', 'pelaku bertanggung', 'suaminya', 'kelurahan', 'srondol', 'kerja pinkan', 'supervisor', 'kecamatan rawalo', 'persebaya', 'minggu rahmat', 'hotel', 'karangtengah', 'hidayatullah', 'sumbersuko', 'sepeda motor', 'jakarta', 'kecelakaan tunggal', 'marabahan', 'daerah pasar', 'rute', 'sanana', 'kerja sehari', 'dilaluinya', 'baleendah kabupaten', 'hati pelaku', 'melewati', 'berkoordinasi', 'mataram', 'rahmat rahmat', 'izin', 'jasad keluarga', 'sangatta']]
Total Kandidat temp:  

In [18]:
#kueri who
kueriAsliWho='siapa pelaku kejadian ini'
kueriWho=preprocessing(kueriAsliWho)
kueriWho= [" ".join (kueriWho)]
print (kueriWho)
hasilkandidatWho=[]
keywordGabungWho=[]
kandidatFixWho=[]
kueriFixWho=[]
hasilDokumenWho=[]


hasilSearchWho=cari_dokpertama(kueriAsliWho)
keywordYakeWho=keyword_yake(hasilSearchWho)
keywordtfidf2Who=keyword_tfidf(hasilSearchWho)
keywordbertWho=keyword_bert (hasilSearchWho)
keywordBOW_Who=keyword_BOW(bow_who_text, kueriAsliWho)

keywordGabungWho.append(keywordYakeWho)
keywordGabungWho.append(keywordtfidf2Who)
keywordGabungWho.append(keywordbertWho)
keywordGabungWho.append(keywordBOW_Who)
hasilrankWho=rangking(keywordGabungWho)

for i in hasilrankWho:
    kueriFixWho.append(i)
for j in kueriFixWho:
    hasilkandidatWho.append(j)
kueriFixWho=[preprocessing(i) for i in kueriFixWho]
for i in kueriFixWho:
    for j in i:
        kandidatFixWho.append(j)
kandidatFixWho= [" ".join (kandidatFixWho)]
print ('*'*120)

['pelaku kejadian']
  (0, 10790)	0.8671961861217088
  (0, 6763)	0.49796664022398396
kandidat temp [['victoria', 'aktivitasnya', 'motivasinya', 'tewas', 'tangerang', 'wanita', 'menghubungkan', 'amy', 'menghubungi keluarga', 'main', 'menghubungi', 'mendekati', 'kejahatan mengejutkan', 'no', 'keterlibatannya', '816', 'mengejutkan kejahatan', 'utara', 'menyelenggarakan', 'menikung', 'kemarin bantuan', 'honda', 'mengurangi', 'brio', 'mengabaikan memutuskan', 'lippo karawaci', 'kendali', 'mengemudikan', 'sumbersuko', 'rumah sakit', 'fahri', 'mengurangi kecepatannya', 'mataram', 'pejabat komisaris', 'melibatkan', 'mengejutkan', 'tebo', 'komisaris patton', 'sepeda', 'keterkaitan keterlibatannya', 'pematangsiantar', 'mobil', 'meninggal', 'mengabaikan', 'temanggung', 'karawaci utara', 'dunia', 'kecelakaan mengakibatkan', 'sunter', 'korban', 'kanit', 'mengeksplorasi', 'malang', 'wanita berinisial', 'madiun', 'pelaku mengeksplorasi', 'maluku', 'berinisial amy', 'ipda', 'mengakibatkan', 'kuin', 'pr

In [19]:
j=1

testing_data = []
for i in range(0, len(document_text_test)-1):
    hasilWhat=[]
    hasilWho=[]
    hasilWhere=[]
    teks=df_test.iloc[i,-2]
    tfidf_vectorizer = joblib.load('vectorizer.pkl')
    tfidf_matrix = tfidf_vectorizer.fit_transform([teks])

    query_vec_What= tfidf_vectorizer.transform(kandidatFixWhat)
    results_what=cosine_similarity(tfidf_matrix, query_vec_What).reshape((-1))
    hasilDokumenWhat.append(df_test.iloc[i,2])
    for a in hasilrankWhat:
        cariW = re.findall(a,hasilDokumenWhat[i])
        #print(cariW)
        if cariW:
            hasilWhat.append(a)
    
    data = [i,df_test.iloc[i,2],'what',kueriAsliWhat, hasilWhat, results_what]

    testing_data.append(data)

    query_vec_Where= tfidf_vectorizer.transform(kandidatFixWhere)
    results_where=cosine_similarity(tfidf_matrix, query_vec_Where).reshape((-1))
    hasilDokumenWhere.append(df_test.iloc[i,2])
    for a in hasilrankWhere:
        cariW = re.findall(a,hasilDokumenWhere[i])
        #print(cariW)
        if cariW:
            hasilWhere.append(a)
    
    data = [i,df_test.iloc[i,2],'where',kueriAsliWhere, hasilWhere, results_where]

    testing_data.append(data)

    query_vec_Who= tfidf_vectorizer.transform(kandidatFixWho)
    results_who=cosine_similarity(tfidf_matrix, query_vec_Who).reshape((-1))
    hasilDokumenWho.append(df_test.iloc[i,2])
    for a in hasilrankWho:
        cariW = re.findall(a,hasilDokumenWho[i])
        #print(cariW)
        if cariW:
            hasilWho.append(a)
    
    data = [i,df_test.iloc[i,2],'who',kueriAsliWho, hasilWho, results_who]

    testing_data.append(data)

    # print (j)
    # print("No ID Dokumen  : ", i)
    # print("Tanggal        : ", df_test.iloc[i,1])
    # print("Isi berita     : ", df_test.iloc[i,2])
    # print ("Hasil W       : ",hasilWhat)
    # print("(Kemiripan: %.4f) " % results_what)
    # print ('*'*120)
    j+=1

In [ ]:
#write to csv
writer = pd.DataFrame(testing_data, columns=['No Document','Description', 'W', 'Kueri W','hasilW', 'Kemiripan'])
writer.to_csv('QE_Stat_testing_result.csv', index=False, sep=',')